# person detection from image, video with Yolov3 et openCV

In [1]:
import cv2
import numpy as np


# 1) Object detection from image

#### Télechargements des poids de Yolov3 et du fichier de configuration de Yolov3

#### Chargement  de Yolo

In [2]:
net = cv2.dnn.readNet("path/yolov3.weights", "yolov3.cfg")


#### Récupération des classes

In [3]:
classes = []
with open('coco.names') as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


#### Chargement de l'image

In [4]:
img = cv2.imread('Path/image.jpeg')
#print(img.shape)
height, width, channels = img.shape

####  Passer l'image dans le réseau et de faire la détection

##### Outs est un tableau qui contient toutes les informations sur les objets détectés, leur position et le degré de confiance de la détection.

In [5]:
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), True, crop = False)
net.setInput(blob)   
outputLayerNames = net.getUnconnectedOutLayersNames()
outs = net.forward(outputLayerNames)

À ce stade, la détection est faite, et il suffit d'afficher le résultat à l'écran.
<br/>on passe ensuite en boucle dans le tableau des résultats, on calcule la confiance et on choisit un seuil de confiance.
<br/>on fixe un seuil de confiance de 0,5, s'il est supérieur, on considère que l'objet est correctement détecté, sinon on le saute.

In [6]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

L'affichages des boites avec l'application d'une fonction pour supprimer ce lebruit "NMSBoxe".
<br/>Si l'objet est une personne on l'affiche sinn on l'affiche pas

In [7]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
colors = np.random.uniform(0, 255, size=(len(classes), 3))

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        if str(classes[class_ids[i]]) == 'person':
            label = str(classes[class_ids[i]])
            confidence = str(round(confidences[i],2))
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label +" "+confidence, (x, y + 30), font, 2, color, 2)
cv2.imshow("Image", img)



cv2.imwrite('Person_detection.jpg', img) 
cv2.waitKey(0)
cv2.destroyAllWindows()
print("The Image was successfully saved") 

The Image was successfully saved


# 1) Object detection from video

###### Le meme principe utilisé pour l'image c-a-d la détection se fait frame par frame sur un video.

In [8]:
cap = cv2.VideoCapture("path/xxxx.mp4")
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
   
size = (frame_width, frame_height) 
   
# Stockage de resulats en videos sous le nom de person_detection. 
result = cv2.VideoWriter('person_detection_video.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size) 
while(True):
    ret, img = cap.read()
    if ret ==True:
    
        height, width, channels = img.shape
        if ret:
            assert not isinstance(img,type(None)), 'frame not found'
        blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), True, crop = False)
        net.setInput(blob)    
        outputLayerNames = net.getUnconnectedOutLayersNames()
        outs = net.forward(outputLayerNames)
        class_ids = []
        confidences = []
        boxes = []

        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        font = cv2.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
               
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i],2))
                color = colors[i]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, label +" "+confidence, (x, y + 30), font, 2, color, 2)


        result.write(img)

        cv2.imshow('Recording...', img)



        if(cv2.waitKey(1) & 0xFF==ord('q')):
            break
    else: 
        break
cap.release() 
result.release() 
    
cv2.destroyAllWindows() 
print("The video was successfully saved") 


The video was successfully saved
